In [1]:
# Import the requisite packages
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import fiona
fiona.supported_drivers

{'DXF': 'rw',
 'CSV': 'raw',
 'OpenFileGDB': 'raw',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'FlatGeobuf': 'raw',
 'GeoJSON': 'raw',
 'GeoJSONSeq': 'raw',
 'GPKG': 'raw',
 'GML': 'rw',
 'OGR_GMT': 'rw',
 'GPX': 'rw',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'S57': 'r',
 'SQLite': 'raw',
 'TopoJSON': 'r'}

In [13]:
# Load Dallas top filers data
df_top_nonpayment = pd.read_csv('data/df_nonpay_100_filer.csv')

In [14]:
# Convert the pandas dataframe to geopandas dataframe
df_top_gpd = gpd.GeoDataFrame(df_top_nonpayment, geometry=gpd.points_from_xy(df_top_nonpayment.X, df_top_nonpayment.Y, crs=4326))

# Dallas County Census Block Group shapefile
data_path = '/Users/wooyongjung/WJ_Projects/eviction_2023_repo/data/tl_2021_48_bg/tl_2021_48_bg.shp'
census_bg = gpd.read_file(data_path)
census_bg = census_bg.to_crs(4326)
census_bg = census_bg[census_bg['GEOID'].str.startswith('48113')]

# Aggregate the eviction locations to the Dallas County census BG
top_bldg_bg = gpd.sjoin(census_bg, df_top_gpd, how='left', op='contains')

/Users/wooyongjung/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [15]:
# Select needed columns
top_bldg_bg = top_bldg_bg[['plaintiff_name','case_number','X','Y','geometry','GEOID']]

In [17]:
# Drop the NaN values in plaintiff_name
top_bldg_bg = top_bldg_bg.dropna(subset=['plaintiff_name'])
top_bldg_bg

,plaintiff_name,case_number,X,Y,geometry,GEOID
120,ALDEN SHORT INC,1798.0,-96.892691,32.930098,"POLYGON ((-96.90265 32.93862, -96.90255 32.938...",481130139012
149,CSH 20161 BORROWER LLC,213.0,-96.943161,32.593518,"POLYGON ((-96.95888 32.59957, -96.95886 32.599...",481130165221
278,SIGNATURE LEASING MGMT INC,311.0,-96.943792,32.580313,"POLYGON ((-96.95973 32.57677, -96.95951 32.576...",481130166263
278,S2 HIGH POINTE LLC,160.0,-96.942595,32.586811,"POLYGON ((-96.95973 32.57677, -96.95951 32.576...",481130166263
346,PLEASANT RUN APARTMENTS,171.0,-96.857752,32.605918,"POLYGON ((-96.87484 32.61063, -96.87480 32.612...",481130166182
...,...,...,...,...,...,...
17904,THE LINK,197.0,-96.732497,32.906481,"POLYGON ((-96.73835 32.90042, -96.73812 32.901...",481130078101
17962,CITY NORTH,307.0,-96.778227,32.925100,"POLYGON ((-96.78633 32.92929, -96.78630 32.930...",481130136103
18060,11760 FERGUSON RD LLC,161.0,-96.649580,32.855095,"POLYGON ((-96.65101 32.85411, -96.64958 32.855...",481130126041
18060,EYAL DALLAS HOLDINGS LLC,648.0,-96.649463,32.855178,"POLYGON ((-96.65101 32.85411, -96.64958 32.855...",481130126041


In [19]:
# Save the aggregated data
top_bldg_bg.to_csv('data/df_nonpay_100_geoid.csv', index=False)